In [1]:
from scipy import stats
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from imageio import imread, imwrite
import random, gc, pickle, os
from PIL import Image
from skimage.transform import resize
from os.path import exists, isfile, join
from math import log
import pandas as pd

# Load Data

In [2]:
def load_folder_list(path=""):
    return [os.path.join(path, o) for o in os.listdir(path) if os.path.isdir(os.path.join(path, o))]

In [3]:
def get_data_path(img_folder):

    img_path = load_folder_list(path = img_folder)
    img_path.sort()

    return img_path

In [4]:
def get_data_name(data_path):

    if exists(data_path):
           data_name = [f for f in os.listdir(data_path) if isfile(join(data_path,f))]
            
    data_name.sort()
    return data_name

In [5]:
def get_data(img_path):

    name = get_data_name(img_path)
    img = []
    
    for p in tqdm(range(len(name))):
        img.append(Image.open(join(img_path, name[p])))
    
    return img, name

In [6]:
img_path = "ClinicalResults/PostImage"
img, img_name = get_data(img_path)

100%|██████████| 59/59 [00:00<00:00, 2085.49it/s]


# Get Fractal Dimension

# Get Probability fractal dimension & mean Lacunarity

In [7]:
def getProbFD(img):
    """
    % GETPROBFD  Compute probability fractal dimension and lacunarity.
    %
    % Input:     img     a gray scale image (2D matrix, 0..255)
    %
    % Output:    FDprob  Probability Fractal Dimension
    %            meanLac Mean Lacunarity over all box sizes
    %
    """

    #if color image is passed reduce to gray
    if (img.shape[2] > 1):
        img = np.sum(img[:, :, 0:3], axis = 2) / 3.0

    #% if range is 0..1, scale to 0 ..255:
    if (np.max(img) <= 1):
        img = 255.0 * img

    img = img.astype(np.uint8)
    h, w = int(img.shape[0]), int(img.shape[1])
    
    lmax = np.min([w, h, 31])

    if (lmax % 2 == 0):
        lmax= lmax-1

    border = int((lmax - 1) / 2)
    results = []
    #print('Calculating over h = ', h, ' and w = ', w)
    #print('l          log(l)                    N(l)                      -log(N(l))')
    #print('=========================================================================')
    count = 0
    L = []
    NL = []
    LacL = []
    totalpix = (h - 2 * border) * (w - 2 * border)
    for l in range(3, lmax+1, 2):
        count = count + 1
        r = int((l-1) / 2)
        Nsum = 0
        Msum = 0
        M2sum = 0

        for y in range(border + 1, h - border + 1):
            for x in range(border + 1, w - border + 1):

                #iterate over box with radius r around x,y
                ibox = np.int16(img[y-r:y+r+1, x-r:x+r+1])
                ibox = np.abs(ibox - np.int16(img[y, x + 1])) <= r
                m = np.sum(ibox)
                Nsum = Nsum + 1.0/m
                Msum = Msum + m
                M2sum = M2sum + m * m

        #print(l, "    ", np.log(l), "     ", Nsum, "      ", -np.log(Nsum))
        L.append(l)
        NL.append(Nsum / totalpix) #division does not change slope below
        M2sum = M2sum / totalpix
        Msum = Msum / totalpix
        #%LacL(count) = (M2sum - Msum^2) / Msum^2;
        LacL.append(M2sum  / (Msum * Msum)) #% the above + 1, we'd add 1 anyway later

    results = []
    results.append(np.log(L))
    results.append(-np.log(NL))
    istart = 2
    iend = len(results[0])

    X = results[0][istart:iend+1]
    Y = results[1][istart:iend+1]
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, Y)
    FDprob = slope
    
    meanLac = np.mean(LacL[istart : iend + 1])
    medianLac = np.median(LacL[istart : iend + 1])

    #print('Probability Fractal Dimension : ', FDprob)
    #print('Mean Probability Lacunarity   : ', meanLac)
    return FDprob, meanLac

In [8]:
def getFD(img):
    """
    % GETFD      Compute fractal dimensions.
    %
    % Input:     img     a gray scale image (2D matrix, 0..255)
    %
    % Output:    FDcap   Capacity Fractal Dimension (box counting)
    %            FDinf   Information Fractal Dimension
    %            FDcor   Correlation Fractal Dimension
    %
    """

    #if color image is passed reduce to gray
    if (img.shape[2] > 1):
        img = np.sum(img[:, :, 0:3], axis = 2) / 3.0

    #% if range is 0..1, scale to 0 ..255:
    if (np.max(img) <= 1):
        img = 255.0 * img

    img = img.astype(np.uint8)
    h, w = img.shape[0], img.shape[1]

    lmax = np.min([w, h, 64])

    wn = int(np.floor(w / lmax) * lmax)
    hn = int(np.floor(h / lmax) * lmax)

    #print('Calculating over h =', hn, 'and w =', wn)
    L = []
    NL = []
    InfL = []
    SqrFreqL = []
    count = 0
    hnwn = hn * wn
    loghw = np.log(hnwn)
    
    for boxsize in [np.power(2, r) for r in range(1, int(np.floor(np.log2(64)))+1)]:
        Nsum = 0
        i = 0
        Inf = 0
        SqrFreq = 0
        bs2 = boxsize * boxsize
        count = count + 1

        for k in range(1, hn-boxsize+2, boxsize):
            for l in range(1, wn-boxsize+2, boxsize):
                ibox = img[k:k+boxsize, l:l+boxsize]
                maxi = np.float32(np.max(ibox))
                mini = np.float32(np.min(ibox))
                N = np.floor((maxi - mini) / boxsize)+1  #number of boxes (round up)
                Nsum = Nsum + N
                Inf = Inf + bs2 * (np.log(bs2 / N) - loghw )
                SqrFreq = SqrFreq + bs2 * bs2 / N
                i = i + 1    # count squares

        #print('Quadrate: ', i)
        #print('N = ', Nsum, ' and -ln(N) = ', -np.log(np.float32(Nsum)), ' for boxsize ', boxsize)
        L.append(boxsize)
        NL.append(Nsum)
        InfL.append(Inf / hnwn)
        SqrFreqL.append(SqrFreq / hnwn * hnwn)

    results = []
    results.append(np.log(L))
    results.append(-np.log(NL))
    results.append(InfL)
    results.append(np.log(SqrFreqL))

    istart = 2
    iend = len(results[0])

    X = results[0][istart:iend+1]
    Y = results[1][istart:iend+1]
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, Y)
    FDcap =slope

    Y = results[2][istart:iend+1]
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, Y)
    FDinf =slope

    Y = results[3][istart:iend+1]
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, Y)
    FDcor =slope

    #print('Capacity Fractal Dimension    : ', FDcap)
    #print('Information Fractal Dimension : ', FDinf)
    #print('Correlation Fractal Dimension : ', FDcor)
                                                                        
    return FDcap, FDinf, FDcor

# Run

In [9]:
# compute probability fractal dimension and mean lacunarity
# compute capacity, information and correlation fractal dimension
for i in tqdm(range(len(img))):
    
    I = (np.array(img[i])/255.0).astype(np.uint8)
    
    # Save Image
    img_savepath = img_path+"/"+img_name[i]
    im = Image.fromarray(I)
    im.save(img_savepath)
    
    # Calculate
    I = I[..., np.newaxis]
    FDprob, meanLac = getProbFD(I)
    FDcap, FDinf, FDcor = getFD(I)
    
    # Generate report
    data = {'Image Name':img_name[i].split('.')[0].split('_')[1], 'Probability Fractal Dimension':[FDprob], 'Capacity Fractal Dimension':[FDcap], 'Information Fractal Dimension':[FDinf], 'Correlation Fractal Dimension':[FDcor], 'Mean Lacunarity':[meanLac]}
    report = pd.DataFrame(data)
    report.to_csv('Report/Post/'+img_name[i].split('.')[0]+'.csv', index = False)

100%|██████████| 59/59 [55:44<00:00, 56.69s/it]
